In [1]:
import os
import sys
root_dir = sys.path[0]

## Prepare Documents
Only needs to be run once, then will persist in memory

In [2]:
# paper from online
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# loader = DirectoryLoader("texts", glob="*.txt", show_progress=True)
# docs = loader.load()

# paper from Ryan
from langchain_community.document_loaders import PyPDFLoader
docs = PyPDFLoader("Instructions/instructions.pdf").load()

In [3]:
raw_text = ''
for i, doc in enumerate(docs):
    text = doc.page_content
    if text:
        raw_text += text
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 100,
    length_function = len,
    is_separator_regex = False,
)
texts = text_splitter.split_text(raw_text)
len(texts)

23

In [4]:
texts[0]

'This is a study in the economics of market decision making. The instructions are simple, and if you follow them carefully and make  \ngood decisions, you might earn a considerable amount of money, which will be paid to you via PayPal.  \nIn this study we are going to simulate a market in which you and the other investors will buy and sell shares in a sequence of \nperiods. You will be in groups of 8 participants for the duration of this study. Before each period you will be provided'

## Prepare Database

In [5]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores.chroma import Chroma
create_db = True # set True to create a new database
model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

embedding_function = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs,

)

In [6]:
db_dir = os.path.join(root_dir, "porter_db")
if create_db:
### Make the chroma and persiste to disk
    db = Chroma.from_texts(texts,
                        embedding_function,
                        persist_directory=db_dir,)
else:
    db = Chroma(persist_directory=db_dir, embedding_function=embedding_function)

In [7]:
### Query the database with 5 most similar documents
query = "When can you accept trades?"

db.similarity_search(query, k=5)

[Document(page_content='must submit an ask that is lower than the current lowest ask. Older bids and asks will remain visible temporarily, but can no longer be  \naccepted.  Only the most recent bid or ask can be accepted. Once there is an acceptance, all old bids and asks are removed.\n6)Your holdings of shares and cash may never go below zero.  Therefore, you cannot offer to sell a share if you do not have one in  your \nholdings.  Similarly, you cannot bid more than the cash in your holdings.'),
 Document(page_content='must submit an ask that is lower than the current lowest ask. Older bids and asks will remain visible temporarily, but can no longer be  \naccepted.  Only the most recent bid or ask can be accepted. Once there is an acceptance, all old bids and asks are removed.\n6)Your holdings of shares and cash may never go below zero.  Therefore, you cannot offer to sell a share if you do not have one in  your \nholdings.  Similarly, you cannot bid more than the cash in your holdi

In [8]:
# create an mmr retriever to get the most relevant matching documents
retriever = db.as_retriever(k=5, fetch_k=20, search_type="mmr")

retriever.get_relevant_documents(query)[1]

Document(page_content='one-half of the investors will receive a signal that rules out one of the two possible outcomes for that period.  Trading and Recording Rules \n1)Anyone wishing to buy a share of a given type is free to submit an offer to buy (called a "bid") a share of a given type at a specified  \nprice. To submit a bid, type in the amount you are offering to pay for a given type of share in the box labeled “Bid.”')

## NORMAL RAG

In [9]:
import textwrap
def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [10]:
from langchain.llms import GPT4All
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser


template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model_path = os.path.join(root_dir,
                          "model",
                          "mistral-7b-instruct-v0.1.Q4_0.gguf")

model = GPT4All(
    model=model_path
)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [13]:
original_query = "What are period earnings if I don't trade?"

In [14]:
text_reply = rag_chain.invoke(original_query)

print(wrap_text(text_reply))

Answer: If you hold x-shares at the end of a period with a value of 100, then your total
x-share earnings in that period would be 5x100 = 500 cash. Similarly, if you hold y-shares
at the end of a period with a value of 0, then your total y-share earnings in that period
would be 4x0 = 0 cash. These are known as "period earnings" and they are calculated based
on the value of the shares you hold at the end of each period. If you don't trade during a
period, these will be your only earnings for that period.


### QUIZ

In [15]:
corect_answer = "c"
question01 = "At the end of each period, share values are: a. 0, b. 100, c. Either 0 or 100"
text_reply = rag_chain.invoke(question01)
print(wrap_text(text_reply))

Answer: c


In [ ]:
question02 = "A person purchased a share of Y for 47 and held the share until the period ended. If the outcome is X, how much money did this person make as a result of the trade?"


## RAG FUSION

In [15]:
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate, PromptTemplate

In [16]:
prompt = ChatPromptTemplate(
    input_variables=["original_query"],
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=[],
                template="You are a helpful assistant that generates multiple search queries based on a single input query.",
            )
        ),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=["original_query"],
                template="Generate multiple search queries related to: {question} \n OUTPUT (4 queries):",
            )
        ),
    ],
)

In [17]:
model = GPT4All(
    model=model_path
)


In [18]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

## RAG Fusion

### Generating Queries

In [19]:
generate_queries = (
    prompt | model | StrOutputParser() | (lambda x: x.split("\n"))
)

In [30]:
generate_queries.invoke({"question": original_query})

['',
 '1. What is the minimum amount of cryptocurrency you can hold without making any trades?',
 '2. Can you calculate my earnings based on not trading in this market?',
 "3. If I don't trade, how much will my portfolio value increase over time?",
 '4. How does not trading affect my overall investment returns?']

## Rank and Fuse

In [21]:
from langchain.load import dumps, loads


def reciprocal_rank_fusion(results: list[list], k=60):
    fused_scores = {}
    for docs in results:
        # Assumes the docs are returned in sorted order of relevance
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    return reranked_results

In [22]:
ragfusion_chain = generate_queries | retriever.map() | reciprocal_rank_fusion

In [23]:
ragfusion_chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'question': {'title': 'Question', 'type': 'string'}}}

In [24]:
ragfusion_chain.invoke({"question": original_query})

C:\Users\Tristan\AppData\Roaming\Python\Python310\site-packages\langchain_core\_api\beta_decorator.py:86: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


[(Document(page_content='held at the end of the period minus the loan \nrepayment. \nThere will be several market periods.  Your PayPal payment will be based on the sum of your earnings from all periods \nplus the $7 payment for showing-up on time for this study.\nIf you have any questions, please raise your hand. \nOtherwise, please remain silent.   \nThe study will begin shortly.'),
  0.04787506400409626),
 (Document(page_content='button.  Similarly, anyone is free to sell a share by accepting someone else’s bid. To sell a share by accepting someone else’s bid, click  \nthe “Sell” button.\n4)When a bid or ask is accepted, a binding contract occurs for a single share.  The contracting parties will have their holdings  \nautomatically updated.\n5)When submitting a bid, you must submit a bid that is higher than the current highest bid.  Similarly, when submitting an ask, you'),
  0.03333333333333333),
 (Document(page_content='This is a study in the economics of market decision making. T

In [25]:
from langchain.schema.runnable import RunnablePassthrough
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

full_rag_fusion_chain = (
    {
        "context": ragfusion_chain,
        "question": RunnablePassthrough()
    }
    | prompt
    | model
    | StrOutputParser()
)

In [26]:
response = full_rag_fusion_chain.invoke({"question": "Tell me about auctions. When can you accept trades?"})
print(wrap_text(response))

Answer: Auctions are a type of market where buyers and sellers come together to trade
goods or services at an agreed-upon price. In this particular auction, you can accept
trades by submitting an ask that is lower than the current highest bid for a given share.
When someone accepts your ask, a binding contract occurs for a single share, and your
holdings will be automatically updated.


In [33]:
response = full_rag_fusion_chain.invoke({"question": "what is experiment payoff for doing nothing?"})
print(wrap_text(response))

Answer: 0
